# Check if CEMT codes are correct
This notebook describes how well the Vaarweg codes are filled in. They are consistently filled in, but only in NL and part of BE. Germany is missing. We also show how to compute a consistent ordinal variable.

This information is based on Vaarweginformatie: https://www.vaarweginformatie.nl/frp/main/#/home

You can get extra info from the EURIS API:
See for example https://developer.eurisportal.eu/docs/reference-data/locks




In [1]:
import pickle

import networkx as nx
import requests

import pandas as pd
import numpy as np


In [2]:
fis_url = 'https://zenodo.org/records/6673604/files/network_digital_twin_v0.3.pickle?download=1'

In [3]:
resp = requests.get(fis_url)

In [4]:
graph = pickle.loads(resp.content)

In [5]:
edges_df = nx.to_pandas_edgelist(graph)

In [11]:
# store nan filled codes under CEMT


def code2cemt(code):
    """convert the almost CEMT like code stored in vaarweginformatie"""
    # Somehow the CEMT class _0 (which is not in the Richtlijn Vaarwegen, but used to be in older versions. We'll replace it by '0'

    
    cemt = ''
    if pd.isna(code):
        cemt = ''
    elif code == '_0':
        # special class defined in richtlijn vaarwegen 2011
        cemt = '0' 
    else:
        cemt = code
    return cemt
    
# We need to add the empty category, otherwise
# add all categories from the richtlijnvaarwegen and add VIIa (not used in NL
categories = ['0', 'I', 'II', 'III', 'IV', 'IVa', 'IVb', 'V_A', 'V_B', 'VI_A', 'VI_B', 'VI_C', 'VII', 'VIIa']
cemt_dtype = pd.CategoricalDtype(categories=categories, ordered=True)

In [7]:
# you can use the above function to create a elegant categorical function, including ordening
# Note that when using the data in the graph the ordening function does not work.
# on a future update we should make the codes uniform and rename them from Code to CEMT
edges_df['CEMT'] = pd.Categorical(edges_df['Code'].apply(code2cemt), dtype=cemt_dtype)

In [12]:
# convert CEMT to M2, with multiple we will select the middle M or upper M class, when no M class exists, pick RWS class from barges. 

cemt2rws_dict = {
    "0": "M0",
    "I": "M1",
    "II": "M2", 
    "III": "M4",  
    "IV": "M7",
    "V_A": "M", 
    "V_B": "BII-2l",
    "VI_A": "M11",
    "VI_B": "BII-4",
    "VI_C": "BII-6l"
}
def cemt2rws(cemt):
    return cemt2rws_dict.get(cemt, '')


categories = ['M0', 'M1', 'BO1', 'M2', 'BO2', 'M3', 'BO3', 'M4', 'BO4', 'M5', 'M6', 'BI', 'M7', 'M8', 'BII-1', 'M9', 'BIIa-1', 'BIIL-1', 'BII-2l', 'M10', 'BII-2b', 'M11', 'M12',  'BII-4', 'BII-6l', 'BII-6b'] 
rws_dtype = pd.CategoricalDtype(categories=categories, ordered=True)

In [13]:
edges_df['rws_richtlijn_vaarwegen'] = pd.Categorical(edges_df['CEMT'].apply(cemt2rws), dtype=rws_dtype)

In [14]:
# update the graph with our new classification info 
for e, info in  graph.edges.items():
    cemt = code2cemt(info['Code'])
    rws = cemt2rws(cemt)
    info['CEMT'] = cemt
    info['rws_richtlijn_vaarwegen'] = rws